# Eval on AFLW2000-Reannotated and AFLW2000 datasets

In [1]:
import pickle
from os.path import join
import gzip
import numpy as np
import scipy.io as sio
import cv2
import torchfile
from tqdm import tqdm

def calcRoiBox(pts):
    bbox = [min(pts[0,:]), min(pts[1,:]), max(pts[0,:]), max(pts[1,:])]
    center = [(bbox[0] + bbox[2]) / 2, (bbox[1] + bbox[3]) / 2]
    radius = max(bbox[2] - bbox[0], bbox[3] - bbox[1]) / 2
    bbox = [center[0] - radius, center[1] - radius, center[0] + radius, center[1] + radius]
    llength = np.sqrt((bbox[2] - bbox[0]) ** 2 + (bbox[3] - bbox[1]) ** 2)
    center_x = (bbox[2] + bbox[0]) / 2
    center_y = (bbox[3] + bbox[1]) / 2
    roi_box = np.zeros((4,1))
    roi_box[0] = np.round(center_x - llength / 2)
    roi_box[1] = np.round(center_y - llength / 2)
    roi_box[2] = roi_box[0] + llength
    roi_box[3] = roi_box[1] + llength
    roi_box = np.round(roi_box).astype(np.int).flatten()
    return roi_box


def ImageROI(img, roi):
    height, width, nChannels = img.shape
    sx = roi[0]
    sy = roi[1]
    ex = roi[2]
    ey = roi[3]
    temp = np.zeros((ey - sy + 1, ex - sx + 1, img.shape[2]))
    if (sx < 0):
        dsx = -sx
        sx = 0
    else:
        dsx = 0

    if (ex >= width):
        dex = temp.shape[1] - (ex + 1 - width) - 1
        ex = width - 1
    else:
        dex = temp.shape[1] - 1

    if (sy < 0):
        dsy = -sy
        sy = 0
    else:
        dsy = 0

    if (ey >= height):
        dey = temp.shape[0] - (ey + 1 - height) - 1
        ey = height - 1
    else:
        dey = temp.shape[0] - 1

        
    roi_box = np.array([sx, sy, ex, ey])
    temp[dsy:dey + 1, dsx:dex + 1, :] = img[sy:ey + 1, sx:ex + 1, :]
   
    roi_img = np.copy(temp)
    return roi_img, roi_box

In [2]:
def calc_nme(pts68_fit_all, pts68_gt_all, roi_boxs, option='re'):
    if option == 'ori':
        pts68_all = pts68_gt_all
    elif option == 're':
        pts68_all = pts68_gt_all
    std_size = 120

    nme_list = []

    for i in range(len(pts68_all)):
        pts68_fit = pts68_fit_all[i]
        pts68_gt = pts68_all[i]

        # build bbox
        minx, maxx = np.min(pts68_gt[0, :]), np.max(pts68_gt[0, :])
        miny, maxy = np.min(pts68_gt[1, :]), np.max(pts68_gt[1, :])
        llength = np.sqrt((maxx - minx) * (maxy - miny))

        #
        dis = pts68_fit - pts68_gt[:2, :]
        dis = np.sqrt(np.sum(np.power(dis, 2), 0))
        dis = np.mean(dis)
        nme = dis / llength
        nme_list.append(nme)

    nme_list = np.array(nme_list, dtype=np.float32)
    return nme_list

def ana(nme_list, yaws_list):
    yaw_list_abs = np.abs(yaws_list)
    ind_yaw_1 = yaw_list_abs <= 30
    ind_yaw_2 = np.bitwise_and(yaw_list_abs > 30, yaw_list_abs <= 60)
    ind_yaw_3 = yaw_list_abs > 60

    nme_1 = nme_list[ind_yaw_1]
    nme_2 = nme_list[ind_yaw_2]
    nme_3 = nme_list[ind_yaw_3]

    mean_nme_1 = np.mean(nme_1) * 100
    mean_nme_2 = np.mean(nme_2) * 100
    mean_nme_3 = np.mean(nme_3) * 100
    # mean_nme_all = np.mean(nme_list) * 100

    std_nme_1 = np.std(nme_1) * 100
    std_nme_2 = np.std(nme_2) * 100
    std_nme_3 = np.std(nme_3) * 100
    # std_nme_all = np.std(nme_list) * 100

    mean_all = [mean_nme_1, mean_nme_2, mean_nme_3]
    mean = np.mean(nme_list) * 100
    std = np.std(nme_list) * 100

    s1 = '[ 0, 30]\tMean: \x1b[32m{:.3f}\x1b[0m, Std: {:.3f}'.format(mean_nme_1, std_nme_1)
    s2 = '[30, 60]\tMean: \x1b[32m{:.3f}\x1b[0m, Std: {:.3f}'.format(mean_nme_2, std_nme_2)
    s3 = '[60, 90]\tMean: \x1b[32m{:.3f}\x1b[0m, Std: {:.3f}'.format(mean_nme_3, std_nme_3)
    # s4 = '[ 0, 90]\tMean: \x1b[31m{:.3f}\x1b[0m, Std: {:.3f}'.format(mean_nme_all, std_nme_all)
    s5 = '[ 0, 90]\tMean: \x1b[31m{:.3f}\x1b[0m, Std: \x1b[31m{:.3f}\x1b[0m'.format(mean, std)

    s = '\n'.join([s1, s2, s3, s5])
    print(s)

    return mean_nme_1, mean_nme_2, mean_nme_3, mean, std

## Eval on AFLW2000-3D

In [3]:
with gzip.open("yaws_list.zip", 'rb') as f:
    buffer = f.read()
files, yaws_list = pickle.loads(buffer)

pred_folder = 'out/AFLW2000-3D'
input_folder = 'data/AFLW2000/'
outputs = []
gt_boxes = []
gt_pts = []
for idx, filename in tqdm(enumerate(files)):
    try:
        with gzip.open(join(pred_folder, filename + '.zip'), 'rb') as f:
            buffer = f.read()
        pred = pickle.loads(buffer)
    except Exception as e:
        print(e)
        yaws_list = np.delete(yaws_list, idx)
        continue
    outputs.append(pred.T)
    pts_file = input_folder + filename[:-4] + '.mat'
    gt = sio.loadmat(pts_file)['pt3d_68']
    gt_pts.append(gt)
    img = cv2.imread(input_folder + filename)
    roi_box = calcRoiBox(gt)
    roi_img, roi_box = ImageROI(img, roi_box)
    gt_boxes.append(np.array(roi_box))
nme_list = calc_nme(outputs, gt_pts, gt_boxes)
ana(nme_list, yaws_list)

2000it [00:25, 78.94it/s]


[ 0, 30]	Mean: 2.286, Std: 1.720
[30, 60]	Mean: 2.903, Std: 1.450
[60, 90]	Mean: 4.318, Std: 5.010
[ 0, 90]	Mean: 2.714, Std: 2.587


(2.2864632308483124,
 2.9029875993728638,
 4.317837208509445,
 2.7143118903040886,
 2.586689405143261)

## Eval on AFLW2000-3D Reannotated

In [4]:
with gzip.open("yaws_list.zip", 'rb') as f:
    buffer = f.read()
files, yaws_list = pickle.loads(buffer)

pred_folder = 'out/AFLW2000-3D-Reannotated/'
input_folder = 'data/AFLW2000-3D-Reannotated/'
outputs = []
gt_boxes = []
gt_pts = []
for idx, filename in enumerate(files):
    pts_file = input_folder + filename[:-4] + '.t7'
    try:
        gt = torchfile.load(pts_file)
    except Exception as e:
        print(e)
        yaws_list = np.delete(yaws_list, idx)
        continue
    gt = gt.T
    gt_pts.append(np.array(gt, dtype=np.float32))
    img = cv2.imread(input_folder + filename)
    roi_box = calcRoiBox(gt)
    roi_img, roi_box = ImageROI(img, roi_box)
    gt_boxes.append(np.array(roi_box))
    try:
        with gzip.open(pred_folder + filename + '.zip', 'rb') as f:
            buffer = f.read()
        pred = pickle.loads(buffer)
    except Exception as e:
        print(e)
        yaws_list = np.delete(yaws_list, idx)
        continue
    outputs.append(pred.T)
nme_list = calc_nme(outputs, gt_pts, gt_boxes)
ana(nme_list, yaws_list)

[Errno 2] No such file or directory: 'data/AFLW2000-3D-Reannotated/image00188.t7'
[Errno 2] No such file or directory: 'data/AFLW2000-3D-Reannotated/image03162.t7'
[ 0, 30]	Mean: 1.592, Std: 2.019
[30, 60]	Mean: 1.755, Std: 0.952
[60, 90]	Mean: 3.156, Std: 4.372
[ 0, 90]	Mean: 1.860, Std: 2.460


(1.591581478714943,
 1.7552752047777176,
 3.1558167189359665,
 1.8601788207888603,
 2.4602575227618217)